In [1]:
from mistral.model import Transformer , ModelArgs
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

import math

In [2]:
import mmap
import random

In [3]:

from transformers import LlamaTokenizerFast

In [4]:
path = "../../mar-tokenizer"

In [6]:
import json

In [100]:
with open('../../mar-tokenizer/tokenizer.json', 'r') as file:
    data = json.load(file)

# Access the 'model' key and its 'vocab' array
vocab_array = data['model']['vocab']

# Count the items inside the 'vocab' array
vocab_count = len(vocab_array)

# Print the 'vocab' array and its count
print("Vocabulary Array:")
print(vocab_array)

print("\nCount of Items in Vocabulary Array:", vocab_count)

Vocabulary Array:
[['<unk>', 0.0], ['<s>', 0.0], ['</s>', 0.0], ['े', -3.198136806488037], ['ी', -3.234046697616577], ['▁', -3.6190669536590576], ['व', -3.791450023651123], ['ा', -3.8097009658813477], ['ल', -3.8552277088165283], ['र', -4.030426979064941], ['▁क', -4.096604347229004], ['.', -4.098381519317627], ['▁आहे', -4.098432540893555], ['न', -4.101080417633057], ['ो', -4.182425022125244], ['श', -4.188457012176514], ['च', -4.234373092651367], ['▁स', -4.389428615570068], [',', -4.434492588043213], ['्या', -4.54027795791626], ['त', -4.6261701583862305], ['ड', -4.655073642730713], ['ही', -4.68520975112915], ['▁केल', -4.7505269050598145], ['ू', -4.913422107696533], ['▁प', -4.955512046813965], ['ाठ', -4.957014083862305], ['म', -5.010734558105469], ['स', -5.011702537536621], ['्य', -5.055701732635498], ['ण', -5.057131290435791], ['ाल', -5.062128067016602], ['क', -5.07785177230835], ['ग', -5.083362102508545], ['▁म', -5.091159343719482], ['▁न', -5.118353843688965], ['▁अ', -5.156129837036133]

In [5]:
device = "cuda"

In [6]:
tokenizer_loaded = LlamaTokenizerFast.from_pretrained(path)

In [11]:
tokens = tokenizer_loaded("डिसेंबर ह्या दिवशी श्रीकृष्णाने अर्जुनाला गीता सांगितली.")
# tokens

In [7]:
arg = ModelArgs(dim=1024,
                n_layers=12,
                head_dim=6,
                hidden_dim=4096,
                n_heads=8,
                n_kv_heads=4,
                norm_eps=1e-6,
                max_batch_size=8,
                vocab_size=200,
                sliding_window=4,
                )


# arg = ModelArgs(dim=1024,
#                 n_layers=6,
#                 head_dim=4,
#                 hidden_dim=4096,
#                 n_heads=6,
#                 n_kv_heads=2,
#                 norm_eps=1e-6,
#                 max_batch_size=8,
#                 vocab_size=200,
#                 sliding_window=4,
#                 )


In [13]:
dim= 1024
n_layers= 6
head_dim= 4
hidden_dim= 512
n_heads= 4
n_kv_heads= 2
norm_eps= 1e-6
vocab_size= 30000

max_batch_size= 2

# For rotary embeddings. If not set, will be infered from sliding window.
rope_theta=  None
# If this is set, use sliding window attention rotating cache.
sliding_window=  None
# If this is set, we will use MoE layers instead of dense layers.
moe=  None

In [8]:
model = Transformer(args=arg).to(device)

In [15]:
for name, param in model.named_parameters():
    print(f"{name}: {param.size()}")

tok_embeddings.weight: torch.Size([200, 1024])
norm.weight: torch.Size([1024])
output.weight: torch.Size([200, 1024])
layers.0.attention.wq.weight: torch.Size([48, 1024])
layers.0.attention.wk.weight: torch.Size([24, 1024])
layers.0.attention.wv.weight: torch.Size([24, 1024])
layers.0.attention.wo.weight: torch.Size([1024, 48])
layers.0.attention_norm.weight: torch.Size([1024])
layers.0.ffn_norm.weight: torch.Size([1024])
layers.0.feed_forward.w1.weight: torch.Size([4096, 1024])
layers.0.feed_forward.w2.weight: torch.Size([1024, 4096])
layers.0.feed_forward.w3.weight: torch.Size([4096, 1024])
layers.1.attention.wq.weight: torch.Size([48, 1024])
layers.1.attention.wk.weight: torch.Size([24, 1024])
layers.1.attention.wv.weight: torch.Size([24, 1024])
layers.1.attention.wo.weight: torch.Size([1024, 48])
layers.1.attention_norm.weight: torch.Size([1024])
layers.1.ffn_norm.weight: torch.Size([1024])
layers.1.feed_forward.w1.weight: torch.Size([4096, 1024])
layers.1.feed_forward.w2.weight: t

In [9]:
model_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
print(f"Model size: {model_size_bytes / (1024**2):.2f} MB")

Model size: 584.41 MB


In [13]:
device

'cuda'

In [8]:
torch.cuda.empty_cache()

In [93]:
import gc


In [94]:
gc.collect()

747

In [114]:
# x = torch.rand(3, 128).unsqueeze(0)
x = torch.rand(2, 2, 128).to(device)

In [116]:
x.shape

torch.Size([2, 2, 128])

In [23]:
tokens = tokenizer_loaded.encode("डिसेंबर ह्या दिवशी श्रीकृष्णाने अर्जुनाला.")

In [ ]:
predicted_text_token = tokenizer_loaded.encode(predicted_text)

In [ ]:
tokens

In [25]:
input_ids, seq_lengths, output_lengths = [], [], []

In [26]:
input_ids.extend(tokens)
seq_lengths.append(len(tokens))

In [16]:
#for inference
# input_ids.extend(tokens)
# seq_lengths.append(len(tokens))

AttributeError: 'list' object has no attribute 'shape'

In [44]:
seq_lengths

[33]

In [30]:
tensor = torch.tensor(input_ids, dtype=torch.long).to(device)

In [42]:
tensor.shape

torch.Size([33])

In [32]:
def sample_top_p(probs: torch.Tensor, p: float):
    assert 0 <= p <= 1

    probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    mask = probs_sum - probs_sort > p
    probs_sort[mask] = 0.0
    probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
    next_token = torch.multinomial(probs_sort, num_samples=1)
    return torch.gather(probs_idx, -1, next_token)


def sample(logits: torch.Tensor, temperature: float, top_p: float):
    if temperature > 0:
        probs = torch.softmax(logits / temperature, dim=-1)
        next_token = sample_top_p(probs, top_p)
    else:
        next_token = torch.argmax(logits, dim=-1).unsqueeze(0)

    return next_token.reshape(-1)

In [33]:
model.eval()

logit = model.forward(input_ids=tensor, seqlens=seq_lengths)

RuntimeError: a Tensor with 6600 elements cannot be converted to Scalar

In [34]:
logit.shape

torch.Size([33, 200])

In [19]:
logit.view(-1, logit.size(-1)).shape

torch.Size([33, 200])

In [24]:
tensor.shape

torch.Size([33])

In [16]:
loss = F.cross_entropy(logit.view(-1, logit.size(-1)), tensor.view(-1), ignore_index=-1)

In [17]:
loss.item()

5.378178596496582

In [23]:
loss.shape

torch.Size([])

In [35]:
scaled_logits = torch.softmax(logit,dim=-1)

In [19]:
scaled_logits.shape

torch.Size([33, 200])

In [36]:
next_token = sample(scaled_logits,temperature=0.5, top_p=0.8)

In [25]:
next_token

tensor([ 64,   3, 165,  28,  91,  48,  67, 166,  70,  15, 125, 116,  88, 102,
         88, 155, 171,  14, 103, 116, 134,  85, 150,  41,  43,  52, 179, 150,
        125,  69, 181, 126,  62], device='cuda:0')

In [39]:
predicted_text = tokenizer_loaded.decode(next_token)

In [38]:
predicted_text

'े मोरासेविडिठे थिंवडॉमुफ मका व ल्रप् त्यां्यानेुरि्यीनष आरकाे'

In [40]:
predicted_text_token = tokenizer_loaded.encode(predicted_text)

In [45]:
input_ids.extend(tokens)
seq_lengths[0] = seq_lengths[0]+len(predicted_text_token)

In [46]:
seq_lengths

[68]

In [47]:
tensor = torch.tensor(input_ids, dtype=torch.long).to(device)

In [48]:
tensor.shape

torch.Size([66])

In [49]:
model.eval()

logit = model.forward(input_ids=tensor, seqlens=seq_lengths)

AssertionError: (68, 66)

In [122]:
loss

tensor(5.6331, device='cuda:0', grad_fn=<NllLossBackward0>)

In [120]:
logit.shape

torch.Size([86, 200])

In [116]:
logit.view(-1, logit.size(-1))

tensor([[ 1.0389e+00,  2.3724e-04,  3.0089e-01,  ..., -1.8550e-02,
          9.4721e-01, -1.9433e+00],
        [ 2.7117e-01, -5.8892e-01, -7.4425e-02,  ..., -5.8432e-01,
          1.7085e-02, -7.2686e-02],
        [-9.9749e-02,  7.0405e-01,  1.1654e-01,  ..., -1.3002e+00,
         -1.7605e+00, -6.0589e-01],
        ...,
        [-1.4768e-01,  4.3750e-01,  5.2466e-01,  ...,  3.8173e-01,
         -1.5919e-01, -7.7899e-01],
        [-7.5604e-01, -7.6008e-01,  6.5404e-01,  ...,  3.9160e-02,
         -5.8079e-01, -8.1597e-02],
        [ 4.1231e-01, -6.0959e-01,  5.8402e-01,  ...,  4.3537e-01,
         -2.5136e-02,  4.9881e-01]], device='cuda:0', grad_fn=<ViewBackward0>)

In [38]:
prob = torch.softmax(logit, dim=-1)

In [40]:
prob.sum()

tensor(43., device='cuda:0', grad_fn=<SumBackward0>)

In [41]:
prob

tensor([[0.0120, 0.0042, 0.0057,  ..., 0.0042, 0.0110, 0.0006],
        [0.0054, 0.0023, 0.0038,  ..., 0.0023, 0.0042, 0.0038],
        [0.0040, 0.0090, 0.0050,  ..., 0.0012, 0.0008, 0.0024],
        ...,
        [0.0033, 0.0060, 0.0065,  ..., 0.0057, 0.0033, 0.0018],
        [0.0022, 0.0022, 0.0090,  ..., 0.0049, 0.0026, 0.0043],
        [0.0065, 0.0023, 0.0077,  ..., 0.0067, 0.0042, 0.0071]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [39]:
prob.shape

torch.Size([43, 200])

In [55]:

def read_text_from_file(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
text = read_text_from_file('D:\mr_dataset\mr.txt')

In [56]:
print(len(text.split()))

174960962


In [ ]:
text

In [10]:
file_path = 'D:\mr_dataset\mr.txt'

In [11]:
tokenizer_loaded.add_special_tokens({'pad_token': '[PAD]'})
tokenizer_loaded.padding_side = "right"
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [13]:
def get_random_chunk(filename):
    
    with open(filename, 'rb') as f:
        block_size = 40
        batch_size=8
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            #data = torch.tensor(tokenizer_loaded.encode(decoded_block), dtype=torch.long)
            data = tokenizer_loaded.encode(decoded_block,padding='max_length' ,max_length=100)
            # data = tokenizer_loaded.encode(decoded_block)
            
    return data #decoded_block

In [13]:
print(len(get_random_chunk(filename='D:\mr_dataset\mr.txt')))

100


In [14]:
print(get_random_chunk(filename='D:\mr_dataset\mr.txt'))

[1, 5, 50, 184, 5, 11, 11, 11, 0, 49, 142, 48, 86, 36, 167, 42, 6, 29, 62, 20, 42, 28, 190, 6, 48, 71, 187, 61, 4, 74, 68, 145, 5, 0, 32, 8, 19, 6, 9, 89, 73, 129, 74, 42, 180, 9, 87, 81, 28, 0, 194, 190, 118, 79, 8, 5, 198, 32, 70, 7, 30, 3, 72]


In [14]:
def get_batch():
    block_size = 40
    batch_size=8
    input_ids, seq_lengths, output_ids= [], [],[]
    
    
    # data = get_random_chunk(filename='D:\mr_dataset\mr.txt')
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    for _ in range(batch_size):
        data = get_random_chunk(filename='D:\mr_dataset\mr.txt')
        input_ids.extend(data[:block_size])
        output_ids.extend(data[1:block_size+1])
        seq_lengths.append(len(data[:block_size]))
    # input_ids.extend([data[i:i+block_size] for i in ix])
    # seq_lengths.append(len(input_ids))
    x = torch.tensor(input_ids, dtype=torch.long).to(device)
   
    # x = torch.stack([data[i:i+block_size] for i in ix])
    # for i in range(batch_size):
    #     output_ids.extend(data[i+1:i+block_size+1])
    # output_ids.extend([data[i+7:i+block_size+7] for i in ix])
    y = torch.tensor(output_ids, dtype=torch.long).to(device)
    
    # y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # x, y = x.to(device), y.to(device)
    #return [data[i:i+block_size] for i in ix],[data[i+7:i+block_size+7] for i in ix]

    # x, y,seq_lengths = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True), seq_lengths.to(device)
    # x, y,seq_lengths = x.to(device, non_blocking=True), y.to(device, non_blocking=True), seq_lengths.to(device)
    return x,y ,seq_lengths

In [14]:
x,y,seq_lengths =get_batch()

In [15]:
x.shape, y.shape , seq_lengths

(torch.Size([320]), torch.Size([320]), [40, 40, 40, 40, 40, 40, 40, 40])

In [149]:
x.dtype

torch.int64

In [36]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    x,y,seq_lengths = ge
    logits = model(input_ids=x, seqlens=seq_lengths)
    loss = F.cross_entropy(logits.view(-1, logits.size(-1)),   y.view(-1), ignore_index=-1)
    
    model.train()
    return out

In [68]:
X, Y = get_batch()
X.shape, Y.shape

(torch.Size([8, 10]), torch.Size([8, 10]))

In [76]:
len(X)

8

In [21]:
logits = model(input_ids=x, seqlens=seq_lengths)

In [25]:
loss = F.cross_entropy(logits.view(-1, logits.size(-1)),   y.view(-1), ignore_index=-1)

In [27]:
loss

tensor(5.4894, device='cuda:0', grad_fn=<NllLossBackward0>)

In [15]:
# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 1000#600000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 200 # how many steps to warm up for
lr_decay_iters = 1000#600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings

In [16]:
# optimizer
optimizer = torch.optim.AdamW(params=model.parameters(), weight_decay= weight_decay, lr=learning_rate, betas=(beta1, beta2))

In [17]:
# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

In [25]:
a = get_lr(33)
a

9.9e-05

In [9]:
model.load_state_dict(torch.load('weights/my-lm_epochs-1000.pth'))

In [20]:
import os

In [21]:
os.environ["PYTORCH_USE_CUDA_DSA"] = "1"

In [26]:
os.environ.get("PYTORCH_USE_CUDA_DSA")

'1'

In [20]:

iter_num= 0
while True:
    total_loss=0
    
    #set lr with deacy_fn for each iter
    lr = get_lr(iter_num) if decay_lr else learning_rate
    x,y,seq_lengths = get_batch()
    logits = model(input_ids=x, seqlens=seq_lengths)
    loss = F.cross_entropy(logits.view(-1, logits.size(-1)),   y.view(-1), ignore_index=-1)
    # print(f'loss: {loss}')
    # total_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
 
    # print(f"epoch: {iter_num:.0f} | Training_loss: {loss:.5f}")
    print(f"epoch: {iter_num:.0f} ")
    iter_num+=1

    if iter_num>max_iters:
        break

# 3. Save the model state dict
MODEL_SAVE_PATH = f'weights/my-lm_epochs-{max_iters}.pth'
print(f'Saving model to: {MODEL_SAVE_PATH}')
torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

    

epoch: 0 
epoch: 1 
epoch: 2 
epoch: 3 
epoch: 4 
epoch: 5 
epoch: 6 
epoch: 7 
epoch: 8 
epoch: 9 
epoch: 10 
epoch: 11 
epoch: 12 
epoch: 13 
epoch: 14 
epoch: 15 
epoch: 16 
epoch: 17 
epoch: 18 
epoch: 19 
epoch: 20 
epoch: 21 
epoch: 22 
epoch: 23 
epoch: 24 
epoch: 25 
epoch: 26 
epoch: 27 
epoch: 28 
epoch: 29 
epoch: 30 
epoch: 31 
epoch: 32 
epoch: 33 
epoch: 34 
epoch: 35 
epoch: 36 
epoch: 37 
epoch: 38 
epoch: 39 
epoch: 40 
epoch: 41 
epoch: 42 
epoch: 43 
epoch: 44 
epoch: 45 
epoch: 46 
epoch: 47 
epoch: 48 
epoch: 49 
epoch: 50 
epoch: 51 
epoch: 52 
epoch: 53 
epoch: 54 
epoch: 55 
epoch: 56 
epoch: 57 
epoch: 58 
epoch: 59 
epoch: 60 
epoch: 61 
epoch: 62 
epoch: 63 
epoch: 64 
epoch: 65 
epoch: 66 
epoch: 67 
epoch: 68 
epoch: 69 
epoch: 70 
epoch: 71 
epoch: 72 
epoch: 73 
epoch: 74 
epoch: 75 
epoch: 76 
epoch: 77 
epoch: 78 
epoch: 79 
epoch: 80 
epoch: 81 
epoch: 82 
epoch: 83 
epoch: 84 
epoch: 85 
epoch: 86 
epoch: 87 
epoch: 88 
epoch: 89 
epoch: 90 
epoch: 91

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
